In [1]:
import os 
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback 
import PyPDF2

/home/sahil/Question_Answ/question/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## in this project i use gemini 

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="")

In [20]:
RESPONSE_JSON = {
   "1": {
        "question": "Explain the impact of the Industrial Revolution on modern society.",
        "marks": 5,
        "expected_answer": {
            "key_points": [
                "Technological advancements",
                "Economic changes",
                "Social changes",
                "Urbanization",
                "Long-term effects on modern society"
            ],
            "length": "150-200 words",
            "answer_template": "The Industrial Revolution had a profound impact on modern society. Technological advancements led to increased productivity and the growth of industries. Economically, it shifted societies from agrarian economies to industrial powerhouses, which in turn caused social changes, including the rise of the working class. Urbanization increased as people moved to cities for jobs, and these changes have had lasting effects on contemporary economies and societal structures."
        }
    },
   "2": {
        "question": "Explain the impact of the Industrial Revolution on modern society.",
        "marks": 5,
        "expected_answer": {
            "key_points": [
                "Technological advancements",
                "Economic changes",
                "Social changes",
                "Urbanization",
                "Long-term effects on modern society"
            ],
            "length": "150-200 words",
            "answer_template": "The Industrial Revolution had a profound impact on modern society. Technological advancements led to increased productivity and the growth of industries. Economically, it shifted societies from agrarian economies to industrial powerhouses, which in turn caused social changes, including the rise of the working class. Urbanization increased as people moved to cities for jobs, and these changes have had lasting effects on contemporary economies and societal structures."
        }
   },
   "3": {
        "question": "Explain the impact of the Industrial Revolution on modern society.",
        "marks": 5,
        "expected_answer": {
            "key_points": [
                "Technological advancements",
                "Economic changes",
                "Social changes",
                "Urbanization",
                "Long-term effects on modern society"
            ],
            "length": "150-200 words",
            "answer_template": "The Industrial Revolution had a profound impact on modern society. Technological advancements led to increased productivity and the growth of industries. Economically, it shifted societies from agrarian economies to industrial powerhouses, which in turn caused social changes, including the rise of the working class. Urbanization increased as people moved to cities for jobs, and these changes have had lasting effects on contemporary economies and societal structures."
        }
    }
   
}

In [34]:
## this template 1 
TEMPLATE="""Text: {text}
You are an expert in creating detailed 5-mark questions and answers. Given the above text, your task is to create a quiz consisting of {number} 5-mark questions and answers for {subject} students in a {tone} tone.
Ensure that:
1. The questions are well-differentiated and not repeated.
2. The questions are directly relevant to the provided text.
3. Each answer contains key points, an expected length, and follows a clear structure.
4. Evaluation criteria for full and partial marks are included, along with feedback for each score level.

Format your response like the RESPONSE_JSON below and use it as a guide. Ensure you create {number} 5-mark questions and answers.

### RESPONSE_JSON"
{RESPONSE_JSON}
"""
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Long Question and Answer Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 70 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_Long Question and Answer:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [22]:
## create the prompt 

quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
) 

## now create a chain for prompt one 

llm_chain1=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz")

In [35]:
## create prompt for second template 

quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
) 

review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review") 

In [36]:
## now use squentail chain 

generate_evaluate_chain=SequentialChain(chains=[llm_chain1, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"])

In [37]:
## now test our model 
file_path="/home/sahil/Question_Answ/data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()

In [38]:
NUMBER=2
SUBJECT="Mechin learning"
TONE="simple"

In [39]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [40]:
quiz=response.get("quiz")

In [41]:
quiz

'### RESPONSE_JSON\n{"1": {"question": "Explain the evolution of machine learning from its early days to modern times. How did the field\'s goals and approaches change over time?", "marks": 5, "expected_answer": {"key_points": ["Early days focused on mimicking human cognitive processes", "Shift from symbolic methods to statistical and probabilistic approaches", "Focus shifted from achieving artificial intelligence to solving practical problems", "Rise of deep learning and its impact on various fields"], "length": "150-200 words", "answer_template": "Machine learning\'s journey has been one of evolution. Initially, it aimed to emulate human cognition, employing symbolic methods and neural networks inspired by the brain. However, the field faced challenges with data acquisition and representation, leading to a shift towards statistical and probabilistic methods. The focus then moved from achieving general AI to tackling practical problems like pattern recognition and information retrieva

In [42]:
quiz=quiz.split("### RESPONSE_JSON\n")[1] 

'{"1": {"question": "Explain the historical development of machine learning, highlighting key milestones and the evolution of its focus.", "marks": 5, "expected_answer": {"key_points": ["Arthur Samuel\'s coining of the term in 1959", "Early models like Samuel\'s checkers program", "Influence of research on human cognitive processes (Hebb, Pitts, McCulloch)", "Development of \'learning machines\' like Cybertron", "Shift in focus from AI to practical problems in the 1990s", "Emphasis on statistical and probabilistic methods"], "length": "150-200 words", "answer_template": "Machine learning has a rich history dating back to the 1950s when Arthur Samuel coined the term and developed a checkers program demonstrating early learning capabilities. Inspired by research on human cognition, early work focused on understanding and mimicking human thought processes. The development of \'learning machines\' like Cybertron in the 1960s furthered this exploration. However, by the 1990s, machine learni

In [43]:
quiz=json.loads(quiz)

In [46]:
quiz_table_data = []
for key, value in quiz.items():
    saq=value["question"] 
    print(saq)
    print("")
    k=(value["expected_answer"]["key_points"])
    quiz_table_data.append({"question": saq,  "Answer": k})

Explain the evolution of machine learning from its early days to modern times. How did the field's goals and approaches change over time?

Explain the concept of 'generalization' in machine learning. How is it related to the bias-variance trade-off, and what are the implications for model performance?



In [48]:
quiz=pd.DataFrame(quiz_table_data)

In [49]:
quiz.to_csv("5ml.csv",index=False)